In [ ]:
# Mount Google Drive if you want to save outputs
#from google.colab import drive
#drive.mount('/content/drive')

# Clone your GitHub repo
!git clone https://github.com/championjeyson/agb-prediction.git

# Move into the repo
%cd agb-prediction

# (Optional) Install dependencies
!pip install -r requirements.txt

# Make sure src/ is importable
import sys
sys.path.append("src")


In [ ]:
# from utils.config import get_config
from inference import run_inference

# CFG = get_config("configs/default.yaml")

# Example: run your inference pipeline
run_inference()
